# UAS PPW

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

csv_path = '/content/drive/MyDrive/Colab Notebooks/dataset_antaranews.csv'
df = pd.read_csv(csv_path)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Tanggal,Penulis,Judul,Artikel,Label
0,"Senin, 11 Desember 2023 23:02 WIB",Pewarta: Laily RahmawatyEditor: Didik Kusbiant...,Polri: Pekan kedelapan Operasi Mantap Brata be...,Jakarta (ANTARA) - Kepolisian Negara Repu...,hukum
1,"Senin, 11 Desember 2023 22:06 WIB",Pewarta: Rangga Pandu Asmara JinggaEditor: Cha...,Mahfud minta masa kerja Satgas BLBI diperpanjang,Realisasinya sudah Rp34 triliun ya dari Rp114...,hukum
2,"Senin, 11 Desember 2023 20:58 WIB",NaN,Rafael Alun Trisambodo dituntut 14 tahun penjara,NaN,hukum
3,"Senin, 11 Desember 2023 20:37 WIB",NaN,"Sidang perdana kasus korupsi KONI Sumsel, terd...",ANTARA - Dua terdakwa tin...,hukum
4,"Senin, 11 Desember 2023 20:20 WIB",NaN,Bawaslu Kota Bogor tertibkan 1.500 APK langgar...,ANTARA - Badan Pengawas P...,hukum
...,...,...,...,...,...
1075,"Kamis, 7 Desember 2023 13:42 WIB",NaN,Operasi SAR erupsi Marapi berakhir hingga peni...,ANTARA - Tiga berita terh...,humaniora
1076,"Kamis, 7 Desember 2023 13:39 WIB",Pewarta: Sean MuhamadEditor: Risbiani Fardania...,"BPJS Kesehatan temukan fraud Program JKN, capa...",Kalau totalnya kan Rp866 miliar ya tahun ini ...,humaniora
1077,"Kamis, 7 Desember 2023 13:34 WIB",Pewarta: Akhyar RosidiEditor: Triono Subagyo ...,Pemkab Lombok Tengah kolaborasi dengan UI di b...,"Praya, Lombok Tengah (ANTARA) - Pemerinta...",humaniora
1078,"Kamis, 7 Desember 2023 13:28 WIB",Pewarta: XinhuaEditor: Desi Purnamawati COPYR...,Nyanyian opera Italia masuk Daftar Warisan Dun...,Roma (ANTARA) - Nyanyian opera Italia din...,humaniora


# Cleaning data  
Tahap membersihkan data dari noise, menghapus
karakter khusus, menghapus angka dan menghilangkan spasi yang
berlebihan. Cleaning data dengan menggunakan library re

In [2]:
!pip install nltk
!pip install Sastrawi
!pip install joblib

In [3]:
import pandas as pd
import nltk
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score
import joblib

In [4]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Missing Value

In [5]:
df.isna().sum()

Tanggal      0
Penulis    194
Judul        1
Artikel     51
Label        0
dtype: int64

In [6]:
data = df.dropna() #menghapus missing value
data.reset_index(drop=True, inplace=True) #atur indeks data mulai dari 0 lagi
data

,Tanggal,Penulis,Judul,Artikel,Label
0,"Senin, 11 Desember 2023 23:02 WIB",Pewarta: Laily RahmawatyEditor: Didik Kusbiant...,Polri: Pekan kedelapan Operasi Mantap Brata be...,Jakarta (ANTARA) - Kepolisian Negara Repu...,hukum
1,"Senin, 11 Desember 2023 22:06 WIB",Pewarta: Rangga Pandu Asmara JinggaEditor: Cha...,Mahfud minta masa kerja Satgas BLBI diperpanjang,Realisasinya sudah Rp34 triliun ya dari Rp114...,hukum
2,"Senin, 11 Desember 2023 20:08 WIB",Pewarta: Jorie MR DarondoEditor: Guido Merung ...,Sulut pertama di Indonesia kukuhkan Gugus Tuga...,Manado (ANTARA) - Sulawesi Utara (Sulut) ...,hukum
3,"Senin, 11 Desember 2023 20:01 WIB",Pewarta: Susanti SakoEditor: Edy M Yakub COPY...,Sengketa pengelolaan Pulau Saronde berlanjut k...,Gorontalo (ANTARA) - Sengketa pengelolaan...,hukum
4,"Senin, 11 Desember 2023 19:49 WIB",Pewarta: Fianda Sjofjan RassatEditor: Chandra ...,KPK akan terus kembangkan penyidikan kasus sua...,"Jadi begini, untuk perkara DJKA, yang pasti K...",hukum
...,...,...,...,...,...
881,"Kamis, 7 Desember 2023 13:48 WIB",Pewarta: Rivan Awal LinggaEditor: M. Hari Atmo...,Desa harus bangun pos pemberdayaan keluarga un...,Jakarta (ANTARA) - Menteri Koordinator Bi...,humaniora
882,"Kamis, 7 Desember 2023 13:39 WIB",Pewarta: Sean MuhamadEditor: Risbiani Fardania...,"BPJS Kesehatan temukan fraud Program JKN, capa...",Kalau totalnya kan Rp866 miliar ya tahun ini ...,humaniora
883,"Kamis, 7 Desember 2023 13:34 WIB",Pewarta: Akhyar RosidiEditor: Triono Subagyo ...,Pemkab Lombok Tengah kolaborasi dengan UI di b...,"Praya, Lombok Tengah (ANTARA) - Pemerinta...",humaniora
884,"Kamis, 7 Desember 2023 13:28 WIB",Pewarta: XinhuaEditor: Desi Purnamawati COPYR...,Nyanyian opera Italia masuk Daftar Warisan Dun...,Roma (ANTARA) - Nyanyian opera Italia din...,humaniora


In [7]:
total = data['Label'].value_counts()
total

metro        339
hukum        276
humaniora    271
Name: Label, dtype: int64

In [8]:
data.shape

(886, 5)

In [9]:
berita = data["Artikel"].astype(str)

In [10]:
berita

0           Jakarta (ANTARA) - Kepolisian Negara Repu...
1       Realisasinya sudah Rp34 triliun ya dari Rp114...
2           Manado (ANTARA) - Sulawesi Utara (Sulut) ...
3           Gorontalo (ANTARA) - Sengketa pengelolaan...
4       Jadi begini, untuk perkara DJKA, yang pasti K...
                             ...                        
881         Jakarta (ANTARA) - Menteri Koordinator Bi...
882     Kalau totalnya kan Rp866 miliar ya tahun ini ...
883         Praya, Lombok Tengah (ANTARA) - Pemerinta...
884         Roma (ANTARA) - Nyanyian opera Italia din...
885         Batam (ANTARA) - Kantor Kesehatan Pelabuh...
Name: Artikel, Length: 886, dtype: object

## Lowecase  
proses untuk mengubah semua huruf dalam dokumen
menjadi huruf kecil (lowercase), yang bertujuan untuk memastikan
konsistensi dalam pemrosesan teks. Case folding dengan lowercase

In [11]:
preprocessing = berita.str.lower()

In [12]:
preprocessing

0           jakarta (antara) - kepolisian negara repu...
1       realisasinya sudah rp34 triliun ya dari rp114...
2           manado (antara) - sulawesi utara (sulut) ...
3           gorontalo (antara) - sengketa pengelolaan...
4       jadi begini, untuk perkara djka, yang pasti k...
                             ...                        
881         jakarta (antara) - menteri koordinator bi...
882     kalau totalnya kan rp866 miliar ya tahun ini ...
883         praya, lombok tengah (antara) - pemerinta...
884         roma (antara) - nyanyian opera italia din...
885         batam (antara) - kantor kesehatan pelabuh...
Name: Artikel, Length: 886, dtype: object

## Tokenisasi  
proses membagi teks menjadi unit-unit terpisah yang
disebut token. Token merupakan unit terkecil dalam teks, yang bisa
berupa kata, frasa, karakter, atau bahkan simbol tertentu. Tujuan dari
tokenisasi adalah untuk mempermudah pemrosesan dan analisis lebih
lanjut terhadap teks. Tokenizing dengan library nltk.tokenize.


In [13]:
def process_tokenize(text):
    text = text.split()
    return text

In [14]:
preprocessing = preprocessing.apply(process_tokenize)
preprocessing

0      [jakarta, (antara), -, kepolisian, negara, rep...
1      [realisasinya, sudah, rp34, triliun, ya, dari,...
2      [manado, (antara), -, sulawesi, utara, (sulut)...
3      [gorontalo, (antara), -, sengketa, pengelolaan...
4      [jadi, begini,, untuk, perkara, djka,, yang, p...
                             ...                        
881    [jakarta, (antara), -, menteri, koordinator, b...
882    [kalau, totalnya, kan, rp866, miliar, ya, tahu...
883    [praya,, lombok, tengah, (antara), -, pemerint...
884    [roma, (antara), -, nyanyian, opera, italia, d...
885    [batam, (antara), -, kantor, kesehatan, pelabu...
Name: Artikel, Length: 886, dtype: object

## Punctuation  
Menghapus seluruh tanda baca yang ada pada konten berita Cleaning data dengan menggunakan library re

In [15]:
def process_punctuation(tokens):
    cleaned_tokens = [re.sub(r'[.,()&=%:-]', '', token) for token in tokens]
    cleaned_tokens = [re.sub(r'\d+', '', token) for token in cleaned_tokens]

    return cleaned_tokens

In [16]:
preprocessing = preprocessing.apply(process_punctuation)
preprocessing

0      [jakarta, antara, , kepolisian, negara, republ...
1      [realisasinya, sudah, rp, triliun, ya, dari, r...
2      [manado, antara, , sulawesi, utara, sulut, men...
3      [gorontalo, antara, , sengketa, pengelolaan, p...
4      [jadi, begini, untuk, perkara, djka, yang, pas...
                             ...                        
881    [jakarta, antara, , menteri, koordinator, bida...
882    [kalau, totalnya, kan, rp, miliar, ya, tahun, ...
883    [praya, lombok, tengah, antara, , pemerintah, ...
884    [roma, antara, , nyanyian, opera, italia, dino...
885    [batam, antara, , kantor, kesehatan, pelabuhan...
Name: Artikel, Length: 886, dtype: object

## stopword  
Tahap menghapus kata-kata yang kurang
berrmakna atau tidak memiliki arti seperti kata: saya, dan, atau. Filtering
stop word dengan library nltk, dengan menggunakan kamus stopword
bahasa Indonesia.


In [17]:
def process_stopword_token(tokens):
    stop_words = set(stopwords.words("indonesian"))
    # custom_stop_words = ['masingmasing','tiaptiap','satusatunya', 'intinya', 'seiring']
    # stop_words.update(custom_stop_words)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return " ".join(filtered_tokens)

In [18]:
preprocessing = preprocessing.apply(process_stopword_token)
preprocessing

0      jakarta  kepolisian negara republik indonesia ...
1      realisasinya rp triliun ya rp triliun rampas r...
2      manado  sulawesi utara sulut provinsi indonesi...
3      gorontalo  sengketa pengelolaan pulau saronde ...
4      perkara djka kpk kembangkan perkara itujakarta...
                             ...                        
881    jakarta  menteri koordinator bidang kesejahter...
882    totalnya rp miliar ya besarjakarta  badan peny...
883    praya lombok  pemerintah kabupaten lombok nusa...
884    roma  nyanyian opera italia dinobatkan warisan...
885    batam  kantor kesehatan pelabuhan kkp kelas i ...
Name: Artikel, Length: 886, dtype: object

## Stemming  
Proses untuk menghapus kata imbuhan untuk mendapatkan
kata dasarnya. Pada proses stemming dengan menggunakan
library sastrawi. PySastrawi adalah sebuah library Python yang
menyediakan fungsi-fungsi untuk melakukan pemrosesan bahasa alami
dalam bahasa Indonesia, seperti stemming (pemenggalan kata ke bentuk
dasarnya).

In [19]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [20]:
preprocessing = preprocessing.apply(lambda text:stemmer.stem(text))
preprocessing

0      jakarta polisi negara republik indonesia laksa...
1      realisasi rp triliun ya rp triliun rampas rp t...
2      manado sulawesi utara sulut provinsi indonesia...
3      gorontalo sengketa kelola pulau saronde camat ...
4      perkara djka kpk kembang perkara itujakarta ko...
                             ...                        
881    jakarta menteri koordinator bidang sejahtera r...
882    total rp miliar ya besarjakarta badan selengga...
883    praya lombok perintah kabupaten lombok nusa te...
884    roma nyanyi opera italia nobat waris budaya ta...
885    batam kantor sehat labuh kkp kelas i batam pul...
Name: Artikel, Length: 886, dtype: object

In [24]:
data['preprocessing_berita'] = preprocessing
data.to_csv('/content/drive/MyDrive/Colab Notebooks/preprocessing_berita.csv', index=False)

<ipython-input-24-9220397b17b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['preprocessing_berita'] = preprocessing


# fitur ekstraksi  



In [25]:
csv_preprocessing = '/content/drive/MyDrive/Colab Notebooks/preprocessing_berita.csv'
datas = pd.read_csv(csv_preprocessing)
datas

,Tanggal,Penulis,Judul,Artikel,Label,preprocessing_berita
0,"Senin, 11 Desember 2023 23:02 WIB",Pewarta: Laily RahmawatyEditor: Didik Kusbiant...,Polri: Pekan kedelapan Operasi Mantap Brata be...,Jakarta (ANTARA) - Kepolisian Negara Repu...,hukum,jakarta polisi negara republik indonesia laksa...
1,"Senin, 11 Desember 2023 22:06 WIB",Pewarta: Rangga Pandu Asmara JinggaEditor: Cha...,Mahfud minta masa kerja Satgas BLBI diperpanjang,Realisasinya sudah Rp34 triliun ya dari Rp114...,hukum,realisasi rp triliun ya rp triliun rampas rp t...
2,"Senin, 11 Desember 2023 20:08 WIB",Pewarta: Jorie MR DarondoEditor: Guido Merung ...,Sulut pertama di Indonesia kukuhkan Gugus Tuga...,Manado (ANTARA) - Sulawesi Utara (Sulut) ...,hukum,manado sulawesi utara sulut provinsi indonesia...
3,"Senin, 11 Desember 2023 20:01 WIB",Pewarta: Susanti SakoEditor: Edy M Yakub COPY...,Sengketa pengelolaan Pulau Saronde berlanjut k...,Gorontalo (ANTARA) - Sengketa pengelolaan...,hukum,gorontalo sengketa kelola pulau saronde camat ...
4,"Senin, 11 Desember 2023 19:49 WIB",Pewarta: Fianda Sjofjan RassatEditor: Chandra ...,KPK akan terus kembangkan penyidikan kasus sua...,"Jadi begini, untuk perkara DJKA, yang pasti K...",hukum,perkara djka kpk kembang perkara itujakarta ko...
...,...,...,...,...,...,...
881,"Kamis, 7 Desember 2023 13:48 WIB",Pewarta: Rivan Awal LinggaEditor: M. Hari Atmo...,Desa harus bangun pos pemberdayaan keluarga un...,Jakarta (ANTARA) - Menteri Koordinator Bi...,humaniora,jakarta menteri koordinator bidang sejahtera r...
882,"Kamis, 7 Desember 2023 13:39 WIB",Pewarta: Sean MuhamadEditor: Risbiani Fardania...,"BPJS Kesehatan temukan fraud Program JKN, capa...",Kalau totalnya kan Rp866 miliar ya tahun ini ...,humaniora,total rp miliar ya besarjakarta badan selengga...
883,"Kamis, 7 Desember 2023 13:34 WIB",Pewarta: Akhyar RosidiEditor: Triono Subagyo ...,Pemkab Lombok Tengah kolaborasi dengan UI di b...,"Praya, Lombok Tengah (ANTARA) - Pemerinta...",humaniora,praya lombok perintah kabupaten lombok nusa te...
884,"Kamis, 7 Desember 2023 13:28 WIB",Pewarta: XinhuaEditor: Desi Purnamawati COPYR...,Nyanyian opera Italia masuk Daftar Warisan Dun...,Roma (ANTARA) - Nyanyian opera Italia din...,humaniora,roma nyanyi opera italia nobat waris budaya ta...


In [26]:
preprocessing_berita = data["preprocessing_berita"]

## TF-IDF  
Pada Tahap ekstraksi fitur, dilakukan dengan metode TF-IDF (Term
Frequency-Inverse Document Frequency) melibatkan perhitungan nilai TF-IDF
untuk setiap kata dalam setiap dokumen. Menghitung TF adalah menghitung
frekuensi kemunculan setiap kata dalam dokumen tertentu. Kemudian
menghitung DF adalah menghitung jumlah dokumen dalam korpus yang
mengandung kata tertentu. Lalu IDF yaitu menghitung kebalikan dari DF, dapat
dihitung dengan menggunakan rumus logaritma dari jumlah total dokumen
dibagi dengan DF untuk kata tersebut. Dan kemudian TF-IDF yaitu mengalikan
nilai TF dengan nilai IDF untuk setiap kata dalam setiap dokumen. Akan
menghasilkan representasi vektor numerik yang memperhitungkan pentingnya
kata dalam dokumen dan dalam korpus secara keseluruhan

In [27]:
tfidfvectorizer = TfidfVectorizer(analyzer='word')
tfidf = tfidfvectorizer.fit_transform(preprocessing_berita)
tfidf_token = tfidfvectorizer.get_feature_names_out()

In [28]:
tfidf_df = pd.DataFrame(data = tfidf.toarray(), columns = tfidf_token)
tfidf_df

,aa,aac,aan,aanberangkat,aandeelhouders,aang,ab,abad,abadi,abai,...,zoom,zoomnya,zubair,zubi,zuhair,zuhdiar,zulfikareditor,zulham,zulherius,zumrotun
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Split Data  
Pembagian dataset dibagi menjadi dua yaitu data training dan data testing. Data training digunakan untuk proses pelatihan. Data testing digunakan untuk testing

In [29]:
X = tfidf_df
y = data['Label']

y

0          hukum
1          hukum
2          hukum
3          hukum
4          hukum
         ...    
881    humaniora
882    humaniora
883    humaniora
884    humaniora
885    humaniora
Name: Label, Length: 886, dtype: object

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
X_train.head()

,aa,aac,aan,aanberangkat,aandeelhouders,aang,ab,abad,abadi,abai,...,zoom,zoomnya,zubair,zubi,zuhair,zuhdiar,zulfikareditor,zulham,zulherius,zumrotun
434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Classification  
Metode Naive Bayes adalah metode klasifikasi yang berdasarkan pada Teorema
Bayes dengan asumsi independensi antara fitur-fitur yang ada. Metode ini cocok untuk digunakan pada data yang memiliki banyak fitur. Pada dasarnya,
algoritma Naive Bayes menghitung probabilitas posterior dari setiap kelas berdasarkan fitur-fitur yang diamati.

In [32]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, y_train)
y_pred_naivebayes = naive_bayes_classifier.predict(X_test)

In [ ]:
cm_naivebayes = confusion_matrix(y_test, y_pred_naivebayes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_naivebayes)
accuracy = accuracy_score(y_test, y_pred_naivebayes)

plt.figure(figsize=(4, 3))
sns.heatmap(cm_naivebayes, annot=True, fmt='d', cmap='Blues')

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix for')
plt.show()

print('Accuracy =', accuracy)

In [ ]:
joblib.dump(naive_bayes_classifier, 'nb_model')

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [36]:
# Inisialisasi model Naive Bayes
rf = RandomForestClassifier()

# Latih model pada kolom topik tertentu
rf.fit(X_train, y_train)

# Lakukan prediksi
y_pred = rf.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred)
print(f'Akurasi model Random Forest: {accuracy_rf}')

Akurasi model Random Forest: 0.9662921348314607


In [39]:
joblib.dump(tfidfvectorizer, 'tfidf_vectorizer')

['tfidf_vectorizer']

In [40]:
joblib.dump(rf, 'random_forest_model')

['random_forest_model']

# Aplikasi Predikisi

Aplikasi untuk klasifikasi berita 👉 [Streamlit]()